In [5]:
import sys
sys.path.append('..')

import torch
from torch import nn
from torch.utils.data import DataLoader
from torchvision import datasets
from torchvision.transforms import ToTensor

from sklearn.preprocessing import MinMaxScaler
from sklearn.model_selection import train_test_split
import pandas as pd
import numpy as np

from utils.timescale_connector import TimescaleConnector
import lightning as L

In [3]:
df = TimescaleConnector.query_ohlcv_daily("ACB")

In [4]:
df.head()

,date,open,high,low,close,volume,symbol
0,2017-01-03,4.43,4.75,4.43,4.75,1609757,ACB
1,2017-01-04,4.75,4.88,4.75,4.78,739000,ACB
2,2017-01-05,4.80,4.83,4.73,4.78,471881,ACB
3,2017-01-06,4.78,5.23,4.73,5.03,1409824,ACB
4,2017-01-09,5.03,5.23,5.03,5.20,970791,ACB


**Define the model with Lightning**